In [39]:
from jupyter_plotly_dash import JupyterDash
    
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64
import os    
import numpy as np
import pandas as pd
from bson.json_util import dumps,loads    
    
from pymongo import MongoClient
from crudMod5 import AnimalShelter    
    
###########################
# Data Manipulation / Model
###########################
#change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
username = 'aacuser1'
password = 'July2021'
shelter = AnimalShelter(username, password)

#update with your username and password and CRUD Python module name

df =pd.DataFrame.from_records(list(dumps(shelter.readAll({}))))

dff = pd.DataFrame.from_records(list(dumps(shelter.readAll({}))))    
    
#########################
#Dashboard Layout / View
#########################
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] 
app = JupyterDash('Dash DataTable Only')
uniqueID = 'AbbyFarnsworth081521'    
    
image_filename = 'Grazioso Salvare Logo.png' #customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard')))
    
    #customer image location
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    #Button location at top
    html.Div(className='row',
        style={'display' : 'flex'},
              children=[
                html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
                html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
                      ]),
    dash_table.DataTable(
        id='datatable-interactivity',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            editable=False,
            #filter_action=native
            sort_action="native",
            sort_mode="multi",
            column_selectable=False,
            row_selectable=False,
            row_deletable=False,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 10,
        ),
        html.Br(),
        html.Hr(),
    
    ])
app.layout = html.Div([
    dcc.RadioItems(
            id='rd',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue ', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Water Rescue',
            labelStyle={'display': 'inline-block'}
        ),
        html.Hr(),
        html.Div(id='dd-output-container', children = [
                dash_table.DataTable(
                    id='datatable-id',
                    columns=[
                        {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                    ])]),
        html.Hr(),
        html.Div(id='map-id'),
        html.Hr(),
        html.Div(id='graph-id')
        ])
@app.callback(
        dash.dependencies.Output('dd-output-container', 'children'),
        [dash.dependencies.Input('rd', 'value')]
        )
def update_output(value):
        df = pd.DataFrame.from_records(shelter.read({}))
        
        if value == 'Water Rescue':
            df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"Labrador Retriever Mix"},
                                                            {"breed":"Chesapeake Bay Retriever"},
                                                            {"breed":"Newfoundland"}
                                                           ],"age_upon_outcome_in_weeks":{'$gt':26,'$lt':156},
                                                        "sex_upon_outcome":"Intact Female"}))
        if value == 'Mountain' or 'Wilderness Rescue':
        
            df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"German Shepherd"},
                                                            {"breed":"Alaskan Malamute"},
                                                            {"breed":"Old English Sheepdog"},
                                                            {"breed":"Siberian Husky"},
                                                            {"breed":"Rottweiler"}  
                                                           ],"age_upon_outcome_in_weeks":{'$gt':26,'$lt':156},
                                                        "sex_upon_outcome":"Intact Male"}))
        if value == 'Disaster Rescue' or 'Individual Tracking':
            df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"German Shepherd"},
                                                            {"breed":"Doberman Pinscher"},
                                                            {"breed":"Golden Retriever"},
                                                            {"breed":"Bloodhound"},
                                                            {"breed":"Rottweiler"}  
                                                           ],"age_upon_outcome_in_weeks":{'$gt':20,'$lt':300},
                                                        "sex_upon_outcome":"Intact Male"}))
            
        return 
        dash_table.DataTable(
                    id='datatable-id',
                    columns=[
                        {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                    ],
                    data=df.to_dict('records'),
                    editable=True,
                    style_cell={'textAlign': 'left'},
                     style_cell_conditional=[
                    {
                        'if': {'column_id': s},
                        'textAlign': 'left'
                    } for s in ['animal_type','color','breed','animal_id']
                ],
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(0, 128, 0)'
                    }
                ],
                style_header={
                    'backgroundColor': 'rgb(0, 238, 0)',
                    'fontWeight': 'bold'
                },
            
                filter_action="native",
                sort_action="native",
                sort_mode="multi",
                column_selectable="single",
                row_selectable="multi",
                row_deletable=True,
                selected_columns=[],
                selected_rows=[],
                page_action="native",
                page_current= 0,
                page_size= 10,
    
                )
                            
    
@app.callback(
        Output('map-id', "children"),
        [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    #Add in the code for your geolocation chart
        dff = pd.DataFrame.from_dict(viewData)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup\
                dl.Marker(position=[30.75,-97.48], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[0,10]),
                        html.H3("Animal Color"),
                        html.P(dff.iloc[0,5]),
                        html.H3("Animal ID"),
                        html.P(dff.iloc[0,2]),
                        
                    ])
                ])
            ])
        ]
            
#############################################
#Interaction Between Components / Controller
#############################################
    
    
@app.callback(Output('datatable-interactivity',"data"), 
                  [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
                  ])
def on_click(bt1,bt2):
        #start case
        if (int(bt1) == 0 and int(bt2) == 0):
            df = pd.DataFrame.from_records(shelter.readAll({}))
        # use higher number of button clicks to determine filter type
        elif (int(bt1) > int(bt2)):
            df = pd.DataFrame(list(shelter.readAll({"animal_type":"Cat"})))
        else (int(bt2) > int(bt1)):
            df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog"})))
        
            
        return df.to_dict('records')
    
#This callback will highlight a row on the data table when the user selects it
"app"



TypeError: object of type 'NoneType' has no len()

[Other notebook](.crudmodule4.ipynb).